# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [13]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [14]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files=glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True )
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
print(dd_px.columns)



Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source',
       'Year'],
      dtype='object')
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume              float64
source               object
Year                  int32
dtype: object


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.


+ Add returns based on Close:

 - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [43]:
# Write your code below.
def add_features(df):
    # sort within ticker by date
    df = df.sort_values(["ticker", "Date"])
    
    # ---- 1. Add lags for Close and Adj Close ----
    df["Close_lag_1"] = df.groupby("ticker")["Close"].shift(1)
    df["Adj_Close_lag_1"] = df.groupby("ticker") ["Adj Close"].shift(1)
    
    # ---- 2. Add returns ----
    df["returns"] = (df["Close"] / df["Close_lag_1"]) - 1
    
    # ---- 3. Add hi_lo_range ----
    df["hi_lo_range"] = df["High"] - df["Low"]
    
    return df

# Apply to each partition in parallel
dd_feat = dd_px.map_partitions(add_features)

dd_feat.compute()


Date       Open       High        Low      Close  Adj Close  \
ticker                                                                     
A      1999-11-18  32.546494  35.765381  28.612303  31.473534  27.068665   
A      1999-11-19  30.713520  30.758226  28.478184  28.880543  24.838577   
A      1999-11-22  29.551144  31.473534  28.657009  31.473534  27.068665   
A      1999-11-23  30.400572  31.205294  28.612303  28.612303  24.607880   
A      1999-11-24  28.701717  29.998211  28.612303  29.372318  25.261524   
...           ...        ...        ...        ...        ...        ...   
ZEUS   2020-03-26   9.610000   9.940000   9.260000   9.590000   9.590000   
ZEUS   2020-03-27   9.330000   9.330000   8.700000   8.700000   8.700000   
ZEUS   2020-03-30   8.810000   9.760000   8.700000   9.680000   9.680000   
ZEUS   2020-03-31   9.640000  10.470000   9.590000  10.350000  10.350000   
ZEUS   2020-04-01  10.140000  10.340000   9.310000   9.470000   9.470000   

            Volume    source  Year  Close_lag_1  Adj_Close_lag_1   returns  \
ticker                                                                       
A       62546300.0     A.csv  1999          NaN              NaN       NaN   
A       15234100.0     A.csv  1999    31.473534        27.068665 -0.082386   
A        6577800.0     A.csv  1999    28.880543        24.838577  0.089783   
A        5975600.0     A.csv  1999    31.473534        27.068665 -0.090909   
A        4843200.0     A.csv  1999    28.612303        24.607880  0.026563   
...            ...       ...   ...          ...              ...       ...   
ZEUS       60500.0  ZEUS.csv  2020     9.670000         9.670000 -0.008273   
ZEUS       52900.0  ZEUS.csv  2020     9.590000         9.590000 -0.092805   
ZEUS       73700.0  ZEUS.csv  2020     8.700000         8.700000  0.112644   
ZEUS       68900.0  ZEUS.csv  2020     9.680000         9.680000  0.069215   
ZEUS       34000.0  ZEUS.csv  2020    10.350000        10.350000 -0.085024   

        hi_lo_range  
ticker               
A          7.153078  
A          2.280043  
A          2.816525  
A          2.592991  
A          1.385908  
...             ...  
ZEUS       0.679999  
ZEUS       0.630000  
ZEUS       1.060000  
ZEUS       0.880000  
ZEUS       1.030000  

[326566 rows x 13 columns]

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

df_feat = dd_feat.compute()


import pandas as pd

df_feat['Date'] = pd.to_datetime(df_feat['Date'])
df_feat = df_feat.sort_values(['ticker', 'Date'])

df_feat['returns_ma_10'] = df_feat.groupby('ticker')['returns'].transform(
    lambda x: x.rolling(window=10, min_periods=1).mean()
)

print(df_feat[[ 'Date', 'returns', 'returns_ma_10']].head(15))


             Date   returns  returns_ma_10
ticker                                    
A      1999-11-18       NaN            NaN
A      1999-11-19 -0.082386      -0.082386
A      1999-11-22  0.089783       0.003698
A      1999-11-23 -0.090909      -0.027837
A      1999-11-24  0.026563      -0.014237
A      1999-11-26  0.003044      -0.010781
A      1999-11-29  0.022762      -0.005191
A      1999-11-30  0.001484      -0.004237
A      1999-12-01  0.017778      -0.001485
A      1999-12-02  0.027656       0.001753
A      1999-12-03  0.008499       0.002427
A      1999-12-06  0.028090       0.013475
A      1999-12-07 -0.010929       0.003404
A      1999-12-08 -0.001381       0.012356
A      1999-12-09  0.013831       0.011083


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

It wasnt necessary to convert to pandas.The calculation can be done in both, but Pandas I think is easier, Dask has more steps with partitions. Also Dask would be more suited for larger data maybe.
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.